In [1]:
#rush hour game methods definition

import numpy as np
#based on gameboard and car letter return x or y
def moveAxis(gameboard,car):
    for char in range (len(gameboard)):
        
        if gameboard[char]==car and 'p1' not in locals():
            p1=char
        elif gameboard[char]==car:  
            p2=char
            break
    if (p1-p2)==-6:
        return 'y'
    if (p1-p2)==-1:
        return 'x'

    
#read text file and return  6x6 array of gameboard and dict of fuel level 
def readfile(filename):
    gameboards=[]
    carFuels=[]
    f = open(filename, "rt")
    count=0
    for x in f:
        gameboard=[]
        carFuel={}
        if x[0]=="#":
            continue
        if len(x)==1:
            continue
        else:
            x=x.strip("\n")
            token=x.split()
            
            for i in range(len(token)):        
                if i==0:
                    count+=1
                    for j in range(36):
                        gameboard.insert(len(gameboard),token[i][j])
                        if token[i][j] not in  carFuel and  token[i][j] !='.':
                            carFuel[token[i][j]]=100
                elif i>=1: 
                    
                    car=token[i][0]
                    fuelLevel=int(token[i][1])
                    carFuel.update({car:fuelLevel})
        gameboard=np.array(gameboard).reshape(6,6)
        gameboards.insert(count-1,gameboard)
        carFuels.insert(count-1,carFuel)        
                          
    f.close()
    return gameboards,carFuels

#adds the valet service to the game
def valetService(gameboard,car):
    valetBoard=gameboard.copy()
    if car=="A":
        return valetBoard
    if valetBoard[17]==car:
        valetBoard[:] = [x if x != car else "." for x in valetBoard]
    return valetBoard    
    
#makes a dict of possible moves
def possiblemoves(gameboard,fuelLevel):
    gameboard=gameboard.flatten().tolist()
    moves={}
    
    for car in fuelLevel.keys():
        if car in gameboard:
            axis=moveAxis(gameboard,car)
            carMoves={}   
            if(axis=='x'):       
                left=MovePossibleLeft(gameboard,car)
                right=MovePossibleRight(gameboard,car)       
                if(left!=0 and fuelLevel[car]>left):
                    carMoves.update({"left":left})
                if(right!=0 and fuelLevel[car]>right):
                    carMoves.update({"right":right})

            if(axis=='y'):
                up=MovePossibleUp(gameboard,car)
                down=MovePossibleDown(gameboard,car)
                if(up!=0 and fuelLevel[car]>up):

                    carMoves.update({"up":up})
                if(down!=0 and fuelLevel[car]>down):
                    carMoves.update({"down":down})

            if(len(carMoves)>0):
                moves.update({car:carMoves})
    return moves

#return spaces car can move left
def MovePossibleLeft(gameboard,car):
    count=0
    index=gameboard.index(car)
    while(index%6 != 0):
        index-=1
        if(gameboard[index]=="."):
            count+=1
            continue
        else:
            break
    return count


#return spaces car can move right
def MovePossibleRight(gameboard,car):
    count=0
    index=gameboard.index(car)
    while(index%6 != 5):
        index+=1
        if(gameboard[index]=="."):
            count+=1
            continue
        if(gameboard[index]==car):
            continue
        else:
            break
    return count


#return spaces car can move Up
def MovePossibleUp(gameboard,car):
    count=0
    index=gameboard.index(car)-6
    while(index>=0):
        
        if(gameboard[index]=="."):
            count+=1
            index-=6
            continue
        else:
            break
    return count

#return spaces car can move down
def MovePossibleDown(gameboard,car):
    count=0
    index=gameboard.index(car)+6
    
    while(index<=35):
        if(gameboard[index]=="."):
            count+=1
            index+=6
            continue
        if(gameboard[index]==car):
            index+=6
            continue
        else:
            break
    return count
#return possible gameboard with possible moves based on a 6x6 gameboard and fuel dict
def possibleboard(gameboard, fuel):
    possiblemove=possiblemoves(gameboard,fuel)
    gameboards=[]
    
    gameboard=gameboard.flatten().tolist()
    
    for car in possiblemove:
        
        for direction in possiblemove[car]:
            count=1
            newBoard=gameboard.copy()
            newFuel=fuel.copy()
            index=newBoard.index(car)
            
            
            if direction=="up":
                while possiblemove[car][direction]>=count:
                    if newBoard[index-6]==".":
                        newBoard[index-6]=car
                        i=1

                        while (newBoard[index+(i*6)]==car)  :
                            i+=1
                            if (index+i*6>35):
                                break
                        newBoard[index+(i*6)-6]='.'
                        option=np.array(newBoard).reshape(6,6)
                        newFuel[car]=newFuel[car]-1
                        gameboards.append((option,newFuel.copy(),direction,count))
                        count+=1
                        index-=6
                        continue
            if direction=="down":
                while possiblemove[car][direction]>=count:
                    
                    newBoard[index]='.'
                    i=1
                    while(newBoard[index+(i*6)]==car) :
                        i+=1
                        if (index+i*6>35):
                            break
                    newBoard[index+(i*6)]=car
                    option=np.array(newBoard).reshape(6,6)
                    newFuel[car]=newFuel[car]-1
                    gameboards.append((option,newFuel.copy(),direction,count))
                    count+=1
                    index+=6
                    continue
            if direction== "left":
                while possiblemove[car][direction]>=count: 
                    newBoard[index-1]=car
                    i=1
                    while(newBoard[index+i]==car) :
                        i+=1
                        if ((index+i)%6)==0:
                            break
                    newBoard[index+i-1]='.'
                    option=np.array(newBoard).reshape(6,6)
                    newFuel[car]=newFuel[car]-1
                    gameboards.append((option,newFuel.copy(),direction,count))
                    count+=1
                    index-=1
                    continue
            if direction=="right":
                while possiblemove[car][direction]>=count:
                    newBoard[index]='.'
                    i=1
                    while (newBoard[index+i]==car) :
                        i+=1
                        if ((index+i)%6)==5:
                            break
                    newBoard[index+i]=car
                    newBoard=valetService(newBoard,car)
                    option=np.array(newBoard).reshape(6,6)
                    newFuel[car]=newFuel[car]-1
                    gameboards.append((option,newFuel.copy(),direction,count))
                    count+=1
                    index+=1
                    continue
    return gameboards
            

In [2]:
#read the file "sample-input.txt"
gameboards,fuel=readfile("sample-input.txt")
#Below prints initial board and fuel level for all initial state in the txt file

for i in range(len(gameboards)):
    print("initial board configuration for board #" , i+1)
    for j in range(len(gameboards[i])):
        print(gameboards[i][j])
    print("Car fuel available", fuel[i])

initial board configuration for board # 1
['B' 'B' 'I' 'J' '.' '.']
['.' '.' 'I' 'J' 'C' 'C']
['.' '.' 'I' 'A' 'A' 'M']
['G' 'D' 'D' 'K' '.' 'M']
['G' 'H' '.' 'K' 'L' '.']
['G' 'H' 'F' 'F' 'L' '.']
Car fuel available {'B': 100, 'I': 100, 'J': 100, 'C': 100, 'A': 100, 'M': 100, 'G': 100, 'D': 100, 'K': 100, 'H': 100, 'L': 100, 'F': 100}
initial board configuration for board # 2
['.' '.' 'I' '.' '.' '.']
['B' 'B' 'I' '.' 'K' '.']
['G' 'H' 'A' 'A' 'K' 'L']
['G' 'H' 'D' 'D' 'K' 'L']
['G' '.' '.' 'J' 'E' 'E']
['F' 'F' '.' 'J' '.' '.']
Car fuel available {'I': 100, 'B': 100, 'K': 100, 'G': 100, 'H': 100, 'A': 100, 'L': 100, 'D': 100, 'J': 100, 'E': 100, 'F': 100}
initial board configuration for board # 3
['J' 'B' 'B' 'C' 'C' 'C']
['J' 'D' 'D' '.' '.' 'M']
['J' 'A' 'A' 'L' '.' 'M']
['F' 'F' 'K' 'L' '.' 'N']
['.' '.' 'K' 'G' 'G' 'N']
['.' 'H' 'H' '.' '.' '.']
Car fuel available {'J': 100, 'B': 100, 'C': 100, 'D': 100, 'M': 100, 'A': 100, 'L': 100, 'F': 100, 'K': 100, 'N': 100, 'G': 100, 'H': 1

In [3]:
%time
#test for functions
#test to return all possible moves based on a board layout and fuel level

#for i in range(len(gameboards)):
    #print(possibleboard(gameboards[i],fuel[i]))
    #print()
       
#test for possiblemove function
#for i in range(len(gameboards)):
    #print("possible moves for initial board in test #", i)
    #print(gameboards[i])
    #print(possiblemoves(gameboards[i],fuel[i]))

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
class Tree:

    def __init__(self,g,gameboard,fuel,direction,count,parent):
        self.g = g
        self.gameboard=gameboard
        self.fuel=fuel
        self.direction=direction
        self.count=count
        self.nodes = []
        self.parent=parent

    def add_node(self, g,gameboard,fuel,direction,count,parent):
        self.nodes.append(Tree(g,gameboard,fuel,direction,count,parent))

    def __repr__(self):
        return f"Tree({self.g,self.gameboard,self.fuel,self.direction,self.count}): {self.nodes}"




In [8]:
def takeFirst(elem):
    return elem[0]

def UCS (gameboard,fuel):
    statesExplored=0
    openq=[(0,gameboard,fuel,None,0,[])]
    closedlist=[]
    #root = Tree(0,gameboard,fuel,None,0,None)
    while openq:
        openq.sort(key=takeFirst)
        currentNode=openq.pop(0)
        
        statesExplored+=1
        gameboard=currentNode[1].flatten()
        fuel=currentNode[2]
        if(gameboard.tolist() in closedlist):
            continue
        if  gameboard[17]=="A":
            return currentNode,statesExplored
        moves=possibleboard(gameboard,fuel);
        closedlist.append(currentNode[1].flatten().tolist())
        for possiblemoves in moves:
            pastStates=currentNode[5].copy()
            board=possiblemoves[0]
            currentfuel=possiblemoves[1]
            direction=possiblemoves[2]
            count=possiblemoves[3]
            g=currentNode[0]+1
            #root.add_node(g,board,currentfuel,direction,count,root)    
            if(board.tolist() not in closedlist):
                pastStates.append((gameboard,fuel,direction,count))
                openq.append((g,board,currentfuel,direction,count,pastStates))                 
    return "No solution",statesExplored 
        
    
for i in range (len(gameboards)):
    print("board #",i)
    %time goalstate=UCS(gameboards[i],fuel[i])
    if(goalstate[0] =="No solution"):
        print (goalstate)
        print ()
    else:
        print("solution path length ",goalstate[0][0])
        print("solution board ",goalstate[0][1])
        print("final fuel level", goalstate[0][2])
        print("direction",goalstate[0][3])
        print("count",goalstate[0][4])
        print(len(goalstate[0][5]))
        for i in range(len(goalstate[0][5])):
                print("move", i+1)
                print("past board",goalstate[0][5][i][0])
                print("past fuelLevel",goalstate[0][5][i][1])
                print("past direction",goalstate[0][5][i][2])
                print("past movecount",goalstate[0][5][i][3])
                print()
        print("search path length ", goalstate[1])
   
    
    
    

board # 0
CPU times: total: 0 ns
Wall time: 999 µs
solution path length  2
solution board  [['B' 'B' 'I' 'J' '.' '.']
 ['.' '.' 'I' 'J' 'C' 'C']
 ['.' '.' 'I' '.' 'A' 'A']
 ['G' 'D' 'D' 'K' '.' 'M']
 ['G' 'H' '.' 'K' 'L' 'M']
 ['G' 'H' 'F' 'F' 'L' '.']]
final fuel level {'B': 100, 'I': 100, 'J': 100, 'C': 100, 'A': 99, 'M': 99, 'G': 100, 'D': 100, 'K': 100, 'H': 100, 'L': 100, 'F': 100}
direction right
count 1
2
move 1
past board ['B' 'B' 'I' 'J' '.' '.' '.' '.' 'I' 'J' 'C' 'C' '.' '.' 'I' 'A' 'A' 'M'
 'G' 'D' 'D' 'K' '.' 'M' 'G' 'H' '.' 'K' 'L' '.' 'G' 'H' 'F' 'F' 'L' '.']
past fuelLevel {'B': 100, 'I': 100, 'J': 100, 'C': 100, 'A': 100, 'M': 100, 'G': 100, 'D': 100, 'K': 100, 'H': 100, 'L': 100, 'F': 100}
past direction down
past movecount 1

move 2
past board ['B' 'B' 'I' 'J' '.' '.' '.' '.' 'I' 'J' 'C' 'C' '.' '.' 'I' 'A' 'A' '.'
 'G' 'D' 'D' 'K' '.' 'M' 'G' 'H' '.' 'K' 'L' 'M' 'G' 'H' 'F' 'F' 'L' '.']
past fuelLevel {'B': 100, 'I': 100, 'J': 100, 'C': 100, 'A': 100, 'M': 99, 'G': 

CPU times: total: 297 ms
Wall time: 296 ms
solution path length  32
solution board  [['E' '.' '.' 'B' 'B' 'H']
 ['E' '.' '.' '.' '.' 'H']
 ['.' '.' '.' '.' 'A' 'A']
 ['C' 'C' 'F' 'G' '.' 'I']
 ['D' 'D' 'F' 'G' '.' 'I']
 ['.' '.' 'F' 'G' '.' 'I']]
final fuel level {'B': 93, 'G': 95, 'H': 98, 'E': 91, 'A': 91, 'I': 99, 'F': 82, 'C': 91, 'D': 94}
direction right
count 4
32
move 1
past board ['B' 'B' '.' 'G' '.' 'H' 'E' '.' '.' 'G' '.' 'H' 'E' 'A' 'A' 'G' '.' 'I'
 '.' '.' 'F' 'C' 'C' 'I' 'D' 'D' 'F' '.' '.' 'I' '.' '.' 'F' '.' '.' '.']
past fuelLevel {'B': 100, 'G': 100, 'H': 100, 'E': 100, 'A': 100, 'I': 100, 'F': 100, 'C': 100, 'D': 100}
past direction right
past movecount 1

move 2
past board ['.' 'B' 'B' 'G' '.' 'H' 'E' '.' '.' 'G' '.' 'H' 'E' 'A' 'A' 'G' '.' 'I'
 '.' '.' 'F' 'C' 'C' 'I' 'D' 'D' 'F' '.' '.' 'I' '.' '.' 'F' '.' '.' '.']
past fuelLevel {'B': 99, 'G': 100, 'H': 100, 'E': 100, 'A': 100, 'I': 100, 'F': 100, 'C': 100, 'D': 100}
past direction up
past movecount 1

move 3
past